In [1]:
import cv2 as cv
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
import tensorflow as tf
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve, auc
from sklearn.cluster import Birch
import seaborn as sns
sns.set()
#pip install pillow 
#pip install opencv-python==3.4.2.16
#pip install opencv-contrib-python==3.4.2.16 --user

Using TensorFlow backend.


In [2]:
imdir = "pnuemoniaimages"

In [3]:
filelist = glob.glob(os.path.join(imdir, '*.jpeg'))

In [4]:
filelist
featurelist = []
descriptors = []
actual_labels = []

In [5]:
 filelist.sort()

In [6]:
#SIFT is only available in pip install --user opencv-contrib-python==3.4.2.17
WIDTH = 250
HEIGHT = 250
for i, imagepath in enumerate(filelist):
    print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    a = 1 if "NORMAL" in imagepath else 0
    actual_labels.append(a)
    img = cv.imread(imagepath)
    d = cv.resize(img, (WIDTH,HEIGHT), interpolation=cv.INTER_CUBIC)
    blur = cv.GaussianBlur(img, (5, 5), 0)
    gray= cv.cvtColor(blur,cv.COLOR_BGR2GRAY)
    sift = cv.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    img=cv.drawKeypoints(gray,kp,outImage = None)
#     cv.imwrite('sift_keypoints.jpg',img)
    kp, des = sift.detectAndCompute(gray,None)
    feature = np.array(des)
    descriptors.append(feature.flatten())

In [7]:
# for i in range(len(descriptors)):
#     descriptors[i] = descriptors[i].flatten()
descriptors

[array([ 1.,  4.,  3., ..., 10.,  0.,  0.], dtype=float32),
 array([2., 0., 0., ..., 4., 3., 2.], dtype=float32),
 array([0., 0., 1., ..., 0., 0., 3.], dtype=float32),
 array([1., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([ 1.,  0.,  0., ...,  0.,  2., 10.], dtype=float32),
 array([0., 0., 0., ..., 0., 1., 2.], dtype=float32),
 array([0., 0., 0., ..., 2., 1., 0.], dtype=float32),
 array([2., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 6., 1., 2.], dtype=float32),
 array([ 0.,  0.,  0., ..., 16.,  0.,  0.], dtype=float32),
 array([ 0.,  0.,  0., ..., 16.,  2., 10.], dtype=float32),
 array([0., 0., 0., ..., 2., 3., 3.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([ 1.,  0.,  0., ..., 41.,  0.,  0.], dtype=float32),
 array([0., 0., 0., ..., 0., 3., 4.], dtype=float32),
 array([ 0.,  0.,  0., ...,  0.,  0., 16.], dtype=float32),
 array([1., 0., 0., ..., 2., 1., 2.], dtype=float32),
 array([ 0.,  0.,  0., ..., 40.,  8.,  2.], dt

In [8]:
#pip install pillow

In [9]:
# featurelist[0:5]
# len(descriptors[5])
# store = [len(x) for x in descriptors]
# store
max_len = np.max([len(a) for a in descriptors])
descriptor = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0.) for a in descriptors])

In [10]:
# featurelist[0][7]
descriptors[0:5]

[array([ 1.,  4.,  3., ..., 10.,  0.,  0.], dtype=float32),
 array([2., 0., 0., ..., 4., 3., 2.], dtype=float32),
 array([0., 0., 1., ..., 0., 0., 3.], dtype=float32),
 array([1., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([ 1.,  0.,  0., ...,  0.,  2., 10.], dtype=float32)]

In [11]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(np.array(descriptor))

In [12]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [13]:
from sklearn.metrics import accuracy_score 
pred = kmeans.labels_
score = round(accuracy_score(pred, actual_labels), 4)
print('Accuracy scored using k-means clustering: ', score)

Accuracy scored using k-means clustering:  0.7766


In [17]:
conf_mat_complete = confusion_matrix(actual_labels, pred)
print("Confusion Matrix")
print(conf_mat_complete)
cr_complete=classification_report(actual_labels, pred)
print(cr_complete)

Confusion Matrix
[[38  2]
 [19 35]]
              precision    recall  f1-score   support

           0       0.67      0.95      0.78        40
           1       0.95      0.65      0.77        54

    accuracy                           0.78        94
   macro avg       0.81      0.80      0.78        94
weighted avg       0.83      0.78      0.78        94



In [18]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

In [19]:
model = AgglomerativeClustering(n_clusters=2,linkage="ward")
model.fit(np.array(descriptor))
model.labels_
x = model.labels_
score = round(accuracy_score(x, actual_labels), 4)
print('Accuracy scored using Agglomerative clustering: ', score)

Accuracy scored using Agglomerative clustering:  0.7128


In [20]:
model1 = AgglomerativeClustering(n_clusters=2, compute_full_tree=True,
    affinity='euclidean', linkage='complete')
model1.fit(np.array(descriptor))
model1.labels_
x1 = model1.labels_
score = round(accuracy_score(x1, actual_labels), 4)
print('Accuracy scored using complete linkage Agglomerative clustering: ', score)

Accuracy scored using complete linkage Agglomerative clustering:  0.4362


In [21]:
model2 = AgglomerativeClustering(n_clusters=2, compute_full_tree=True,
    affinity='euclidean', linkage='single')
model2.fit(np.array(descriptor))
model2.labels_
x2 = model2.labels_
score = round(accuracy_score(x2, actual_labels), 4)
print('Accuracy scored using single linkage Agglomerative clustering: ', score)

Accuracy scored using single linkage Agglomerative clustering:  0.4362


In [22]:
bclust=Birch(branching_factor=100, threshold=.5,n_clusters=2).fit(np.array(descriptor))
print(bclust)
labels_birch = bclust.predict(np.array(descriptor))
labels_birch
score = round(accuracy_score(labels_birch, actual_labels), 4)
print('Accuracy scored using birch clustering: ', score)

Birch(branching_factor=100, compute_labels=True, copy=True, n_clusters=2,
      threshold=0.5)
Accuracy scored using birch clustering:  0.7128
